In [1]:
import os
import pandas as pd
import scanpy as sc
from anndata import AnnData
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib.pyplot import rc_context
from matplotlib import cm

In [2]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.set_theme(style="ticks", rc=custom_params)

# matplotlib.rcParams['figure.figsize'] = [5, 5]
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=900, color_map='viridis_r')
%matplotlib inline

os.chdir('/mnt/data/hong/2022/human_placenta/')


2024-03-22 13:25:15.673964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


scanpy==1.9.4 anndata==0.8.0 umap==0.5.5 numpy==1.26.3 scipy==1.11.4 pandas==1.5.3 scikit-learn==1.4.0 statsmodels==0.14.0 igraph==0.10.3 pynndescent==0.5.8


In [3]:
ad_all = sc.read_h5ad('output/10x_h5/h5ad/ad_v0.9.h5ad')

In [8]:
pd.crosstab(ad_all.obs['sample'], ad_all.obs['subset'], normalize='index')*100

subset,B cells,CTB,EVT,Endo-1,Endo-2,Fibroblast,Hofbauer cells,Monocyte,Myofibroblast,NK cells,STB,T cells,eSTB,pCTB
sample,,,,,,,,,,,,,,
placenta_25,0.096556,15.030576,1.963309,6.276151,1.480528,8.786611,14.805278,0.354039,3.411651,1.190859,42.516897,0.225298,1.287416,2.574831
placenta_32,0.684932,11.907271,0.000000,9.325606,1.106428,8.587987,9.430980,0.737619,3.582719,1.844046,51.264489,0.421496,0.000000,1.106428
placenta_60,0.094073,8.297272,1.467545,5.061148,1.185325,3.762935,3.198495,0.432738,2.031985,0.696143,70.780809,0.131703,0.733772,2.126058
placenta_81,0.100756,4.408060,0.931990,6.498741,0.881612,5.037783,3.853904,0.528967,1.183879,0.730479,74.256927,0.302267,0.226700,1.057935
placenta_248,0.142735,10.191265,0.171282,11.989723,2.255210,14.359121,7.479303,0.342564,4.110762,0.742221,45.732230,0.085641,0.542392,1.855552
placenta_303,0.050556,11.931244,0.101112,1.693630,0.151668,2.426694,2.401416,0.682508,0.581395,0.353893,74.671385,0.075834,1.061678,3.816987
placenta_306,0.513875,7.091470,0.000000,5.070230,1.267557,3.014731,4.453580,0.548133,3.699897,0.993491,69.784173,0.445358,1.027749,2.089757
placenta_312,0.000000,6.829680,0.000000,7.082631,2.192243,3.372681,2.107926,0.843170,2.107926,0.843170,71.838111,0.252951,0.843170,1.686341
placenta_314,0.000000,4.090909,0.000000,0.909091,1.818182,0.454545,3.636364,0.000000,0.000000,0.000000,89.090909,0.000000,0.000000,0.000000


In [22]:
data = pd.read_csv('data/celltype_proportion.tsv', sep='\t', decimal=',')

In [23]:
data_stb = data.filter(regex='group|^STB')
## wide to long, two var names, group and cell type
data_stb2plot = (data_stb.set_index(['group'])
   .rename_axis(['state'], axis=1)
   .stack()
   .reset_index())

In [24]:
data_stb2plot.rename(columns={0: "proportion"}, inplace=True)

In [25]:
data_stb2plot['proportion']=data_stb2plot['proportion'].astype(float)

In [35]:
data_stb2plot['group'].astype('category').cat.reorder_categories(['N-AGA', 'O-AGA', 'O-LAGA'], inplace=True)

/tmp/ipykernel_1857473/2248960175.py:1: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  data_stb2plot['group'].astype('category').cat.reorder_categories(['N-AGA', 'O-AGA', 'O-LAGA'], inplace=True)


In [38]:
sns.catplot(
    data=data_stb2plot, x="group", y="proportion", col="state", errorbar=('pi', 50), order=['N-AGA', 'O-AGA', 'O-LAGA'],
     kind="bar",height=4, aspect=.6,
)
plt.savefig('figures/celltype_proportion_STB.pdf')